In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import dspy

In [4]:
import mlflow
mlflow.set_tracking_uri("http://localhost:5001")
mlflow.set_experiment("test")
mlflow.dspy.autolog()


2025/12/01 18:59:31 WARNING mlflow.utils.autologging_utils: MLflow dspy autologging is known to be compatible with 2.5.22 <= dspy, but the installed version is 3.1.0b1. If you encounter errors during autologging, try upgrading / downgrading dspy to a compatible version, or try upgrading MLflow.


In [5]:
import random
from dspy.datasets import DataLoader

kwargs = dict(fields=("claim", "supporting_facts", "hpqa_id", "num_hops"), input_keys=("claim",))
hover = DataLoader().from_huggingface(dataset_name="vincentkoc/hover-parquet", split="train", trust_remote_code=True, **kwargs)

hpqa_ids = set()
hover = [
    dspy.Example(claim=x.claim, titles=list(set([y["key"] for y in x.supporting_facts]))).with_inputs("claim")
    for x in hover
    if x["num_hops"] == 3 and x["hpqa_id"] not in hpqa_ids and not hpqa_ids.add(x["hpqa_id"])
]

random.Random(0).shuffle(hover)
trainset, devset, testset = hover[:100], hover[100:200], hover[650:]

example = trainset[0]

print("Claim:", example.claim)
print("Pages that must be retrieved:", example.titles)

import ujson
import bm25s
import Stemmer
import os
from dspy.utils import download

# Download and extract data if not exists
if not os.path.exists("wiki.abstracts.2017.jsonl"):
    download("https://huggingface.co/dspy/cache/resolve/main/wiki.abstracts.2017.tar.gz")
    os.system("tar -xzvf wiki.abstracts.2017.tar.gz")

corpus = []
if os.path.exists("wiki.abstracts.2017.jsonl"):
    with open("wiki.abstracts.2017.jsonl") as f:
        for line in f:
            line = ujson.loads(line)
            corpus.append(f"{line['title']} | {' '.join(line['text'])}")

stemmer = Stemmer.Stemmer("english")
corpus_tokens = bm25s.tokenize(corpus, stopwords="en", stemmer=stemmer)

retriever = bm25s.BM25(k1=0.9, b=0.4)
retriever.index(corpus_tokens)

DOCS = {}

def search(query: str, k: int) -> list[str]:
    tokens = bm25s.tokenize(query, stopwords="en", stemmer=stemmer, show_progress=False)
    results, scores = retriever.retrieve(tokens, k=k, n_threads=1, show_progress=False)
    
    retrieved_docs = [corpus[doc] for doc in results[0]]
    
    for doc_str in retrieved_docs:
         if " | " in doc_str:
             title, text = doc_str.split(" | ", 1)
             DOCS[title] = text
             
    return retrieved_docs

# search("France", 5)

def search_wikipedia(query: str) -> list[str]:
    """Returns top-5 results and then the titles of the top-5 to top-30 results."""

    topK = search(query, 30)
    titles, topK = [f"`{x.split(' | ')[0]}`" for x in topK[5:30]], topK[:5]
    return topK + [f"Other retrieved pages have titles: {', '.join(titles)}."]

def lookup_wikipedia(title: str) -> str:
    """Returns the text of the Wikipedia page, if it exists."""

    if title in DOCS:
        return DOCS[title]

    results = [x for x in search(title, 10) if x.startswith(title + " | ")]
    if not results:
        return f"No Wikipedia page found for title: {title}"
    return results[0]

def top5_recall(example, pred, trace=None):
    gold_titles = example.titles
    recall = sum(x in pred.titles[:5] for x in gold_titles) / len(gold_titles)

    # If we're "bootstrapping" for optimization, return True if and only if the recall is perfect.
    if trace is not None:
        return recall >= 1.0
    
    # If we're just doing inference, just measure the recall.
    return recall

evaluate = dspy.Evaluate(devset=devset, metric=top5_recall, num_threads=16, display_progress=True, display_table=5)

instructions = "Find all Wikipedia titles relevant to verifying (or refuting) the claim."
signature = dspy.Signature("claim -> titles: list[str]", instructions)
react = dspy.ReAct(signature, tools=[search_wikipedia, lookup_wikipedia], max_iters=20)

/Users/isaac/Projects/dspy-4-v0/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Claim: This director is known for his work on Miss Potter. The Academy of Motion Picture Arts and Sciences presents the award in which he was nominated for his work in "Babe".
Pages that must be retrieved: ['Academy Award for Best Director', 'Miss Potter', 'Chris Noonan']


In [6]:
from dspy.datasets import DataLoader
from datasets import load_dataset


student_lm = dspy.LM("openrouter/qwen/qwen3-8b")
teacher_lm = dspy.LM("openai/gpt-5")
dspy.configure(lm=student_lm)

In [7]:
react(claim="David Gregory was born in 1625.").titles[:3]

['David Gregory (physician)']

Trace(trace_id=tr-698ea6bca4b70e16af6d1579d7fb10cd)

In [8]:
def safe_react(claim: str):
    try:
        return react(claim=claim)
    except Exception as e:
        return dspy.Prediction(titles=[])

eval_result = evaluate(safe_react)
print(eval_result)


Average Metric: 58.33 / 100 (58.3%): 100%|██████████| 100/100 [00:06<00:00, 16.29it/s]

2025/12/01 19:02:35 INFO dspy.evaluate.evaluate: Average Metric: 58.3333333333333 / 100 (58.3%)


,claim,example_titles,trajectory,reasoning,pred_titles,_patched
0,The Church of England's movement that inspired the Trinity Episcop...,"[Samuel Rickards, Oxford Movement, Trinity Episcopal Church (Hough...","{'thought_0': ""I need to verify if the Trinity Episcopal Church in...","The claim links the Trinity Episcopal Church in Houghton, Michigan...","[Trinity Episcopal Church (Houghton, Michigan), Oxford Movement, S...",✔️ [1.000]
1,"Red, White & Crüe and this athlete both fight. The french fighter ...","[Red, White &amp; Crüe, Mike Tyson, Bobby Stewart]",{'thought_0': 'I need to identify the French fighter trained by Bo...,"The claim conflates a music album (""Red, White & Crüe"" by Mötley C...","[Red, White & Crüe, Mötley Crüe, French Stewart, Bobby Nash, Fight...",✔️ [0.000]
2,The writer/director/actor from Glen or Glenda and Fernand Rivers s...,"[Fernand Rivers, Ed Wood, Glen or Glenda]",{'thought_0': 'I need to verify the claim by checking the careers ...,"The claim suggests that the individuals associated with ""Glen or G...","[Glen or Glenda, Ed Wood, Fernand Rivers, Berlingot and Company, T...",✔️ [1.000]
3,The film by Sandi Sissel was released before The End of Suburbia.,"[The End of Suburbia, Chicken Ranch (film), Sandi Sissel]",{'thought_0': 'I need to determine the release dates of Sandi Siss...,"The claim states that a film by Sandi Sissel was released before ""...","[Chicken Ranch, Paul Jacobs and the Nuclear Gang, Mother Teresa (1...",✔️ [0.333]
4,The actor who played captain hook in the live production with Tayl...,"[Taylor Louderman, Peter Pan Live!, Christopher Walken]",{'thought_0': 'I need to identify the actor who played Captain Hoo...,"The actor who played Captain Hook in the 2014 ""Peter Pan Live!"" pr...","[Peter Pan Live!, Christopher Walken, The Deer Hunter]",✔️ [0.667]


EvaluationResult(score=58.33, results=<list of 100 results>)


[Trace(trace_id=tr-4f96eb4d86bf390d0e8aa8bc57ead008), Trace(trace_id=tr-05c65038847f74f3da1b69cc09458fe8), Trace(trace_id=tr-120d0e7a5952fb3df06cbdf1cb267ff3), Trace(trace_id=tr-d5a8d71d8217aabf7f104275a6388249), Trace(trace_id=tr-bec33846f70d5e9cc7f54dc42ce21536), Trace(trace_id=tr-9fef5821c040ba88a2969dad18594435), Trace(trace_id=tr-4886f5078de7726c9fe89fdf0f7f38af), Trace(trace_id=tr-2df6897de3831662db629272468bc3dc), Trace(trace_id=tr-0c582f4b728931096c3bf9f246b48812), Trace(trace_id=tr-ff86245575c85b681fa00d907cc01279)]

In [25]:
mlflow.dspy.autolog(disable=True)

In [14]:
import dspy

from dspy.teleprompt.teleprompt import Teleprompter
from typing import Any

class NoLabelOptimizer(Teleprompter):
    def compile(self, student: dspy.Module, *, trainset: list[dspy.Example], teacher_lm: dspy.LM) -> dspy.Module:

        def wrap_student(student_prog):
            def wrapped_student(*args, **kwargs):
                with dspy.context(trace=[]):
                    result = student_prog(*args, **kwargs)
                    trace = dspy.settings.trace
                    return [result, trace]
            return wrapped_student
        evaluate = dspy.Evaluate(devset=trainset, metric=lambda e, p, t=None: 0, num_threads=16, display_progress=True)
        eval_result = evaluate(wrap_student(student))
        print(eval_result)

        return student, eval_result

    def get_params(self) -> dict[str, Any]:
        """
        Get the parameters of the teleprompter.

        Returns:
            The parameters of the teleprompter.
        """
        return self.__dict__

optimizer = NoLabelOptimizer()

optimized_react, eval_result = optimizer.compile(react, trainset=trainset[:20], teacher_lm=teacher_lm)


Average Metric: 0.00 / 10 (0.0%):  45%|████▌     | 9/20 [00:00<00:08,  1.33it/s]

/Users/isaac/Projects/dspy-4-v0/.venv/lib/python3.11/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## ne...tailed information.\n'}), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...finish_reason': 'stop'}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
/Users/isaac/Projects/dspy-4-v0/.venv/lib/python3.11/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## ne...o verify the claim.\n'}), input_type=

Average Metric: 0.00 / 11 (0.0%):  55%|█████▌    | 11/20 [00:36<00:30,  3.37s/it]

/Users/isaac/Projects/dspy-4-v0/.venv/lib/python3.11/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## re...tles in the output.\n"}), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...finish_reason': 'stop'}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
/Users/isaac/Projects/dspy-4-v0/.venv/lib/python3.11/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## ne...e titles collected.\n'}), input_type=

Average Metric: 0.00 / 12 (0.0%):  60%|██████    | 12/20 [00:41<00:28,  3.59s/it]

...

/Users/isaac/Projects/dspy-4-v0/.venv/lib/python3.11/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## re...t people and shows.\n'}), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...finish_reason': 'stop'}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


Average Metric: 0.00 / 20 (0.0%): : 22it [03:01,  8.26s/it]

/Users/isaac/Projects/dspy-4-v0/.venv/lib/python3.11/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## re...de those two bands.\n'}), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...finish_reason': 'stop'}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
2025/12/02 01:23:00 INFO dspy.evaluate.evaluate: Average Metric: 0 / 20 (0.0%)



EvaluationResult(score=0.0, results=<list of 20 results>)


[Trace(trace_id=tr-195a63448f8bd3a56a03628f7891cddc), Trace(trace_id=tr-850c876a64c57b6f20b3872422f77bd5), Trace(trace_id=tr-7b73bc18ea32e6b28e1102c2f57782d5), Trace(trace_id=tr-35f54805580b8c9dd59ebd45ca2f66fd), Trace(trace_id=tr-22638b206785cba65709e2c0a6ae1f45), Trace(trace_id=tr-ed23af8131aab5bbbc2dc9147b84470d), Trace(trace_id=tr-ef53e267ba4861d8b192e45f3444902e), Trace(trace_id=tr-2bfe0123d50fe2071616e96c0423e39e), Trace(trace_id=tr-5dc2119c0dfceb1af4a2e36fea149024), Trace(trace_id=tr-64de314c0b79d85a59a3a932f5ad723e)]

In [15]:
from collections import Counter
predictions = [result[1] for result in eval_result.results]
print(type(predictions))
print(type(predictions[0]))
print(len(predictions[0]))
print(predictions[0][1])
c = Counter(type(prediction) for prediction in predictions)
len_c = Counter(len(prediction) for prediction in predictions)
traces = [prediction[1] for prediction in predictions]
final_trajectories = [trace[-1] for trace in traces]
print(final_trajectories[0])
# print(type(traces[0]))


<class 'list'>
<class 'list'>
2
[(Predict(StringSignature(claim, trajectory -> next_thought, next_tool_name, next_tool_args
    instructions="Find all Wikipedia titles relevant to verifying (or refuting) the claim.\n\nYou are an Agent. In each episode, you will be given the fields `claim` as input. And you can see your past trajectory so far.\nYour goal is to use one or more of the supplied tools to collect any necessary information for producing `titles`.\n\nTo do this, you will interleave next_thought, next_tool_name, and next_tool_args in each turn, and also when finishing the task.\nAfter each tool call, you receive a resulting observation, which gets appended to your trajectory.\n\nWhen writing next_thought, you may reason about the current situation and plan for future steps.\nWhen selecting the next_tool_name and its next_tool_args, the tool must be one of:\n\n(1) search_wikipedia, whose description is <desc>Returns top-5 results and then the titles of the top-5 to top-30 result

In [35]:
import rich
class ReasonAboutTraces(dspy.Signature):
    """Given a set of trajectories, and the prompt that was used to generate them,
    reason about the traces and how you might improve the prompt from a strategy perspective without overfitting"""
    trajectories: str = dspy.InputField()
    old_prompt: str = dspy.InputField()
    deviations_from_specification: str = dspy.OutputField(desc="Deviations from the specification given in the signature of the prompt")
    noticed_patterns: str = dspy.OutputField()
    generic_strategy: str = dspy.OutputField()
    improved_prompt: str = dspy.OutputField()

inputs = {
    "trajectories": final_trajectories,
    "old_prompt": signature.instructions,
}
# input_range = [1,3,5,10,20]

# all_range_prompts = {}

# for i in input_range:
#     all_range_prompts[i] = {
#         "trajectories": final_trajectories[:i],
#         "old_prompt": signature.instructions,
#     }

# all_range_prompts = {}
# with dspy.context(lm=teacher_lm):
#     for i in input_range:
#         inputs = all_range_prompts[i]
#         prompt = reasoner(**inputs).improved_prompt
#         all_range_prompts[i] = prompt
#         rich.print(prompt)

all_20_prompts = {}

with dspy.context(lm=teacher_lm):
    for i in range(5):
        print("Running ", i)
        inputs = {
            "trajectories": final_trajectories[:20],
            "old_prompt": signature.instructions,
        }

        reasoner = dspy.Predict(ReasonAboutTraces, seed=i)
        prompt = reasoner(**inputs).improved_prompt
        all_20_prompts[i] = prompt
        rich.print(prompt)



    


Running  0


You are given a claim. Your task is to find all Wikipedia page titles that are relevant to verifying or refuting 
the claim.

Operate with the following approach and tool contract:

1) Decompose the claim
- Identify all named entities (people, works, organizations, places) and the relationships between them (awards, 
roles, locations, counts, dates).
- For pronouns like “this group/this director/that plane,” first resolve the referent by querying for the most 
distinctive clue in the claim (e.g., a song/film title, route terminus, specific award).

2) Retrieve and confirm pages
- Use search_wikipedia with {"query": "<keywords>"} to find candidate pages for each entity and for any verifying 
context (e.g., award pages, lines/stations, ecoregions).
- From search results, select the most likely canonical Wikipedia title (e.g., add disambiguators like “(film)”, 
“(TV series)”, “(band)”, “(director)”, year-qualified titles).
- Confirm the selected title with lookup_wikipedia using {"title": "<exact page title>"}.
- If lookup_wikipedia returns no page:
  - Retry search with alternate spellings, synonyms, year variations, or disambiguation qualifiers.
  - Do not anchor to an unverified year; prefer the page that actually exists (e.g., “Lionheart (1990 film)” vs 
“(1988 film)”).

3) Build the titles list
- Include the canonical titles for:
  - Each primary entity mentioned or implied in the claim.
  - Key verifying context pages needed to check the relationship (e.g., an award page, an organization’s page, a 
specific route/terminus, an airport page with passenger stats).
- Only include titles that you actually observed in search results or successfully confirmed via lookup_wikipedia 
in this run.
- Use exact Wikipedia casing/punctuation; avoid made-up or non-canonical labels (e.g., use “Gracie Humaitá”, not 
“Gracie Humaitá school”).
- Deduplicate the list.

4) Keep reasoning concise
- In reasoning, briefly explain how the gathered titles tie back to the elements of the claim and their 
verification, without overexplaining or asserting facts not supported by the pages you found.

5) Tool usage rules (strict)
- search_wikipedia: {"query": "<keywords>"}.
- lookup_wikipedia: {"title": "<exact page title>"}.
- finish: must be called with {} only (no arguments). Do not pass “reason”, “title(s)”, or any other keys to 
finish.

6) Error recovery and quality
- If a lookup fails, refine the search (try alternative queries, disambiguation, or synonyms) before concluding.
- When “current” facts are involved (e.g., band lineup counts), rely on the “current members” section of the page; 
include the relevant titles even if the status is time-sensitive.

Outputs
- reasoning: A short, step-by-step note of how the titles you collected map to the claim’s entities/relations.
- titles: A deduplicated list of canonical Wikipedia titles you used/confirmed that are relevant to verifying or 
refuting the claim.

Call finish with {} when done.

Running  1


Your task: Find all Wikipedia titles relevant to verifying (or refuting) the claim.

Follow these guidelines:
- Think step by step, but be concise. Begin the reasoning text with: “Reasoning: Let’s think step by step in order 
to …”
- Parse the claim to extract:
  - Concrete anchors (e.g., named films/songs/books/awards/lines/places).
  - People and roles (director, actor, minister, editor).
  - Key dates/numbers that help disambiguate.
  - Anaphora like “this group/person”; resolve these via the distinctive anchor (e.g., the single or film 
mentioned).

Search and verify:
- Use search_wikipedia to find likely pages. Include qualifiers like (film), (TV series), (song), (film director), 
or years to disambiguate.
- Use lookup_wikipedia to confirm canonical titles and facts.
- If a lookup fails:
  - Refine the query (add role/parenthetical/year, adjust capitalization/spelling).
  - Alternatively, open the work’s page (film/song/TV series) and navigate to the person/organization from there.

Choose titles carefully:
- Include only Wikipedia page titles that are directly necessary to verify/refute the claim.
- Cover all sides of the relationship:
  - People + their works (film/song/book) + organizations/awards/roles/places referenced.
- Ensure titles exist and are canonical (use the exact page title as shown on the page).
- Use correct disambiguation and year (e.g., “Lionheart (1990 film)”, “Babe (film)”).
- Do not include non-existent or guessed titles. If a title is not found, search again and adjust.
- Keep the list focused and deduplicated.

Output format:
- reasoning: a short paragraph starting with “Reasoning: Let’s think step by step in order to …” explaining how the
selected pages verify/refute the claim.
- titles: a list of canonical Wikipedia page titles relevant to verification/refutation.

Tool constraints:
- When finished, call finish with empty arguments {} only. Do not pass any keys like title, titles, reason, or type
to finish.

Running  2


You are given a short claim. Your job is to find all Wikipedia page titles that are relevant to verifying or 
refuting the claim.

Output fields:
- Reasoning: Brief, step-by-step explanation of how you decomposed the claim into entities and which pages are 
needed to check the claim.
- Titles: A deduplicated list of canonical Wikipedia page titles that, together, are sufficient to verify or refute
the claim.

Tool usage rules:
- Use search_wikipedia(query: str) to find candidate pages.
- Use lookup_wikipedia(title: str) to confirm a page’s exact, canonical title and that it exists.
- If a lookup fails, refine the query (consider alternate spellings, disambiguation, year variants) and try again. 
Do not include non-existent pages.
- Resolve ambiguous references (e.g., “this group/figure/director”) by first identifying the concrete entity via a 
search on a distinguishing term (e.g., a song/film/TV series name).
- Include both endpoints and any essential bridging pages (e.g., film + award/festival; railway line + terminus 
town; airline + acquiring airline + HQ airport).
- Prefer the most specific pages that directly contain the needed facts.
- Strictly call finish with no arguments (i.e., finish()) when you are done. Never pass any arguments to finish.

Formatting requirements:
- Reasoning: concise bullet points or short sentences describing the entities and why the selected titles are 
needed.
- Titles: a JSON-like array of strings containing only canonical Wikipedia page titles (e.g., ["Chris Noonan", 
"Babe (film)", "Academy Award for Best Director"]). Do not invent titles; ensure each exists via lookup or a clear 
search result.

Constraints and quality checks:
- Do not make a final truth judgment; your task is to assemble the set of titles needed to verify or refute the 
claim.
- Avoid irrelevant list/disambiguation pages unless necessary to resolve ambiguity.
- If a page name in your notes doesn’t resolve, adjust and confirm the correct canonical title before adding it 
(e.g., “Lionheart (1990 film)” instead of “Lionheart (1988 film)”; “Gracie Humaitá” instead of “Gracie Humaitá 
school”; “Vintage Life” not “Vintage Life Magazine”).
- Keep the Titles minimal yet sufficient; deduplicate exact repeats.

Running  3


You are given a claim. Your task is to find all Wikipedia page titles that are directly relevant to verifying or 
refuting the claim, then output:
- reasoning: a brief step-by-step explanation of how the selected titles would verify/refute the claim.
- titles: a minimal list of canonical Wikipedia page titles needed to check the claim.

Follow these rules:
- Tools:
  - search_wikipedia(query: str): use for discovery with descriptive queries (include years, disambiguators like 
“(film)”, locations, etc.).
  - lookup_wikipedia(title: str): use to confirm the exact canonical page title exists and to extract precise facts
(dates, roles, locations).
  - finish(): call with no arguments when done. Do not pass any arguments to finish.

- Disambiguation and validation:
  - Prefer canonical titles with correct disambiguation, e.g., “D.C. Cab (film)”, “Lionheart (1990 film)”, 
“Willunga, South Australia”.
  - After selecting a candidate from search, confirm it exists with lookup before adding it to titles.
  - If lookup fails, refine the query/title (adjust year, add/remove disambiguators, alternate spelling) and retry.

- Title selection:
  - Include both endpoints and necessary connectors. Examples: person + work; work + award/organization; place + 
line/station; group + single.
  - Keep the list minimal and relevant; avoid extraneous or invented titles. Do not include pages that you could 
not confirm with lookup.
  - Resolve pronouns like “this group/film/person” by first identifying the referenced entity, then proceed.

- Reasoning:
  - Be concise and evidence-focused. Indicate which facts are confirmed from which titles.
  - You do not need to declare the claim true/false; just ensure the titles you return would let a reader 
verify/refute it.

- Termination:
  - When you have gathered and validated the minimal set of titles, output reasoning and titles, then call finish()
with no arguments.

Running  4


You are given a claim. Your task is to find all Wikipedia page titles that are relevant to verifying or refuting 
that claim. Produce a brief reasoning explaining the verification plan and list the exact, canonical Wikipedia 
titles needed.

Available tools and required arguments:
- search_wikipedia: {"query": "<string>"} — Use for broad or disambiguated searches. Add qualifiers like "(film)", 
"(TV series)", "(album)", "(song)", "(person)" as appropriate. Include years or other descriptors to disambiguate.
- lookup_wikipedia: {"title": "<exact page title>"} — Use when you already know the exact canonical page title from
a search result.
- finish: {} — Call only when you have collected all necessary titles. Do not pass any arguments to finish.

Process:
1) Parse the claim. Identify key entities, works, awards, organizations, locations, dates.
2) Plan what pages you need to verify each component.
3) Use search_wikipedia to find precise page titles. If a lookup fails or a title is ambiguous, refine your search 
(add qualifiers such as domain, year, nationality, aliases, or “director/author/album/film/song/species”).
4) Use lookup_wikipedia only with exact titles found via search.
5) Title hygiene: Only output existing, exact Wikipedia titles (correct casing and parentheticals). Do not 
fabricate titles. If a title you wrote isn’t found, refine and re-search.
6) Coverage: Include all and only the pages strictly needed to verify/refute the claim components. Avoid peripheral
pages unless they provide necessary verification.
7) If the claim is time-sensitive (e.g., “current members”), still collect the authoritative titles (e.g., the band
page) instead of asserting a verdict.
8) When done, output:
   - Reasoning: a concise plan and what each title helps verify.
   - Titles: a list of exact Wikipedia page titles.
9) Call finish with {}. Never pass arguments to finish.

Notes and tips:
- For films/songs/albums/series with common names, always use the disambiguated form, e.g., “Babe (film)”, “Anne 
(TV series)”.
- For alternate titles or year discrepancies, check disambiguation pages and international release info (e.g., 
“Lionheart (1990 film)” aka “Wrong Bet”).
- If search results include helpful related pages (e.g., known horror films by a director), use them to triangulate
the correct person/title.

Output fields to produce before calling finish:
- reasoning: A concise explanation of how the collected titles verify/refute the claim.
- titles: A list of exact Wikipedia page titles (strings).

Example tool usage (JSON):
- search_wikipedia: {"query": "Rhinestone (film) director"}
- lookup_wikipedia: {"title": "Bob Clark"}
- finish: {}

In [19]:
import warnings
warnings.filterwarnings("ignore", message=r"(?s)Pydantic serializer warnings:.*StreamingChoices")


In [ ]:
evaluate_trainset = dspy.Evaluate(devset=trainset[:10], metric=top5_recall, num_threads=16, display_progress=True)
evaluate = dspy.Evaluate(devset=devset, metric=top5_recall, num_threads=16, display_progress=True, max_errors=50)

# new_signature = signature.with_instructions(outputs.improved_prompt)
# improved_react = dspy.ReAct(new_signature, tools=[search_wikipedia, lookup_wikipedia], max_iters=10)

initial_react = dspy.ReAct(signature, tools=[search_wikipedia, lookup_wikipedia], max_iters=10)
no_cache_lm = dspy.LM("openrouter/qwen/qwen3-8b", cache=None)

def make_react_safe(program):
    def safe_program(*args, **kwargs):
        try:
            return program(*args, **kwargs)
        except Exception as e:
            print(f"Error in program: {e}")
            return dspy.Prediction(titles=[])

    return safe_program

results = {}
with dspy.context(lm=student_lm):
    # initial_react_evaluate = evaluate_trainset(initial_react)
    # improved_react_evaluate = evaluate_trainset(improved_react)
    
    # print(initial_react_evaluate.score)
    # print(improved_react_evaluate.score)
    initial_full_evaluate = evaluate(make_react_safe(initial_react))
    results[-1] = initial_full_evaluate.score

    import concurrent.futures

    def evaluate_prompt_pair(num, prompt):
        new_signature = dspy.Signature("claim -> titles: list[str]", prompt)
        new_react = dspy.ReAct(new_signature, tools=[search_wikipedia, lookup_wikipedia], max_iters=10)
        new_full_evaluate = evaluate(make_react_safe(new_react))
        return num, new_full_evaluate.score

    # Prefer ThreadPoolExecutor for notebook compatibility; ProcessPoolExecutor can misbehave in notebooks
    with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
        # Dictionary mapping future to num for tracking and ordering
        future_to_num = {
            executor.submit(evaluate_prompt_pair, num, prompt): num 
            for num, prompt in all_20_prompts.items()
        }
        for future in concurrent.futures.as_completed(future_to_num):
            num, score = future.result()
            results[num] = score

print("="*80)
rich.print("Original prompt score: ", results[-1])
results.pop(-1)
rich.print("New prompt scores: ", results)
rich.print("Average score: ", sum(results.values()) / len(results))


Average Metric: 61.00 / 100 (61.0%): 100%|██████████| 100/100 [00:09<00:00, 10.84it/s]

2025/12/02 14:44:57 INFO dspy.evaluate.evaluate: Average Metric: 60.99999999999998 / 100 (61.0%)



Average Metric: 0.67 / 1 (66.7%):   0%|          | 0/100 [00:02<?, ?it/s]





Average Metric: 0.67 / 1 (66.7%):   1%|          | 1/100 [00:02<03:25,  2.08s/it]



Average Metric: 1.33 / 2 (66.7%):   1%|          | 1/100 [00:02<03:25,  2.08s/it]



Average Metric: 1.33 / 2 (66.7%):   2%|▏         | 2/100 [00:02<02:02,  1.25s/it]





Average Metric: 1.67 / 3 (55.6%):   2%|▏         | 2/100 [00:03<02:02,  1.25s/it]





Average Metric: 1.67 / 3 (55.6%):   3%|▎         | 3/100 [00:03<01:28,  1.10it/s]





Average Metric: 2.33 / 4 (58.3%):   3%|▎         | 3/100 [00:03<01:28,  1.10it/s]





Average Metric: 2.33 / 4 (58.3%):   4%|▍         | 4/100 [00:04<01:18,  1.22it/s]





Average Metric: 3.00 / 5 (60.0%):   4%|▍         | 4/100 [00:05<01:18,  1.22it/s]





Average Metric: 3.00 / 5 (60.0%):   5%|▌         | 5/100 [00:05<01:28,  1.07it/s]





Average Metric: 3.67 / 6 (61.1%):   5%|▌         | 5/100 [00:05<01:28,  1.07it/s]





Average Metric: 3.67 / 6 (61.1%):   6%|▌         | 6/1

2025/12/02 14:45:31 INFO dspy.evaluate.evaluate: Average Metric: 61.3333333333333 / 100 (61.3%)



Average Metric: 61.33 / 100 (61.3%): 100%|██████████| 100/100 [00:30<00:00,  3.31it/s]

...

2025/12/02 14:45:31 INFO dspy.evaluate.evaluate: Average Metric: 62.66666666666664 / 100 (62.7%)


2025/12/02 14:45:31 INFO dspy.evaluate.evaluate: Average Metric: 54.99999999999999 / 100 (55.0%)


Original prompt score:  61.0

New prompt scores: 
{2: 61.33, 1: 61.33, 3: 61.0, 0: 62.67, 4: 55.0}

Average score:  60.266

In [ ]:
with dspy.context(lm=student_lm):
    initial_evaluation = evaluate(initial_react)
    